In [1]:
import warnings
warnings.filterwarnings('ignore')
import hoomd
import gsd
import matplotlib.pyplot as plt
import numpy as np
import gsd.hoomd
from flowermd.utils import get_target_box_number_density
import unyt as u
import hoomd

## System Initialization

In [3]:
from flowermd.base import Pack,Lattice, Simulation
from flowermd.library import EllipsoidChain

ellipsoid_chain = EllipsoidChain(lengths=1,num_mols=10,lpar=1.0,bead_mass=1.0)
system = Pack(molecules=ellipsoid_chain, density=0.05*u.Unit("nm**-3"), packing_expand_factor=6,edge=2,overlap=1,fix_orientation=True)

## Forcefield definition

In [4]:
from flowermd.library import EllipsoidForcefield

ff = EllipsoidForcefield(epsilon=1.0,lpar=2.0,lperp=1.0,r_cut=2.5,bond_k=None,bond_r0=None)

## Rigid Body Constraint

In [8]:
from flowermd.utils.constraints import create_rigid_ellipsoid_chain

rigid_frame, rigid = create_rigid_ellipsoid_chain(system.hoomd_snapshot)

## Simulation

In [9]:
del ellipsoid_sim

In [10]:
gsd_path=('ellipsoid-chain-2mer.gsd')
ellipsoid_sim = Simulation(
    initial_state=rigid_frame,
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    dt=0.001,
    gsd_write_freq=int(1e4),
    gsd_file_name=gsd_path,
    log_write_freq=int(1e4),
    log_file_name='log.txt')

target_box = get_target_box_number_density(density=0.05*u.Unit("nm**-3"),n_beads=20)
ellipsoid_sim.run_update_volume(final_box_lengths=target_box, kT=7.0, n_steps=1e5,tau_kt=100*ellipsoid_sim.dt,period=10,thermalize_particles=True)
print("shrink finished")
ellipsoid_sim.run_NVT(n_steps=1e5, kT=1.0, tau_kt=10*ellipsoid_sim.dt)
#ellipsoid_sim.save_restart_gsd("restart.gsd")
ellipsoid_sim.flush_writers()
#ellipsoid_sim.save_simulation("sim.pickle")

Initializing simulation state from a gsd.hoomd.Frame.


RuntimeError: Cannot place <hoomd.md.bond.Harmonic object at 0x28d268530> into two simulations.

## Function to redefine rigid constraint for restarting simulation

In [6]:
def restart_rigid_ellipsoid():
    local_coords = [(0.0, 0.0, 0.0), (1.049999999999999, 0.0, 0.0), (1.0, 0.0, 0.0), (-1.0000000000000009, 0.0, 0.0)]
    rigid_constrain = hoomd.md.constrain.Rigid()
    rigid_constrain.body["R"] = {
        "constituent_types": ["X", "A", "T", "T"],
        "positions": local_coords,
        "orientations": [[1, 0, 0, 0]] * len(local_coords),
    }
    return rigid_constrain

In [7]:
ff = EllipsoidForcefield(epsilon=1.0,lpar=1.0,lperp=0.5,r_cut=2.0,bond_k=100,bond_r0=0,angle_k=30,angle_theta0=1.9)
rigid = restart_rigid_ellipsoid_sim()
gsd_path = "sim_restart.gsd"

sim = Simulation(
    initial_state="restart.gsd",
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    gsd_write_freq=int(1e3),
    gsd_file_name=gsd_path,
    log_write_freq=int(1e4),
    log_file_name='log2.txt',
)
sim.run_NVT(n_steps=1e5, kT=1.0, tau_kt=10*sim.dt)
sim.flush_writers()

NameError: name 'restart_rigid_ellipsoid_sim' is not defined

## Adding ellispoid information to GSD 

In [16]:
import gsd.hoomd
gsd_path=('ellipsoid-chain-2mer.gsd')
ellipsoid_gsd(gsd_file=gsd_path,new_file="ovito-ellipsoid.gsd",ellipsoid_types='R',lpar=1.0,lperp=0.5)

In [11]:
def ellipsoid_gsd(gsd_file, new_file, ellipsoid_types, lpar, lperp):
    """Add needed information to GSD file to visualize ellipsoids.

    Saves a new GSD file with lpar and lperp values populated
    for each particle. Ovito can be used to visualize the new GSD file.

    Parameters
    ----------
    gsd_file : str
        Path to the original GSD file containing trajectory information
    new_file : str
        Path and filename of the new GSD file
    ellipsoid_types : str or list of str
        The particle types (i.e. names) of particles to be drawn
        as ellipsoids.
    lpar : float
        Value of lpar of the ellipsoids
    lperp : float
        Value of lperp of the ellipsoids

    """
    with gsd.hoomd.open(new_file, "w") as new_t:
        with gsd.hoomd.open(gsd_file) as old_t:
            for snap in old_t:
                shape_dicts_list = []
                for ptype in snap.particles.types:
                    if ptype == ellipsoid_types or ptype in ellipsoid_types:
                        shapes_dict = {
                            "type": "Ellipsoid",
                            "a": lpar,
                            "b": lperp,
                            "c": lperp,
                        }
                    else:
                        shapes_dict = {"type": "Sphere", "diameter": 0.001}
                    shape_dicts_list.append(shapes_dict)
                snap.particles.type_shapes = shape_dicts_list
                snap.validate()
                new_t.append(snap)